In [ ]:
%%bash
# Colab-specific setup
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install yacs 2>&1 >> install.log
pip install torchviz >> install.log
pip install gdown >> install.log
git init 2>&1 >> install.log
git remote add origin https://github.com/CSAILVision/semantic-segmentation-pytorch.git 2>> install.log
git pull origin master 2>&1 >> install.log
DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log
wget http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet101-upernet/decoder_epoch_50.pth >> install.log
wget http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet101-upernet/encoder_epoch_50.pth >> install.log
rm data/training.odgt
rm data/validation.odgt
gdown --id 1bzMACxZYpTLorDG5E25w7WVSzqRw_e6I
gdown --id 1vJxPY1EJNuL5LZbOnPCHvf7XdzUp7Rf-
gdown --id 1JxHfcIfni3vzsjDQehKsUrpKhuSfWzso
unzip ade20k.zip
mkdir data/ade20k
mkdir data/ade20k/images
mkdir ckpt/ade20k-resnet101-upernet
mv ade20k/images/training data/ade20k/images/training
mv ade20k/images/validation data/ade20k/images/validation
mv training.odgt data/training.odgt
mv validation.odgt data/validation.odgt

From https://github.com/CSAILVision/semantic-segmentation-pytorch
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
Archive:  ade20k.zip
   creating: ade20k/images/
   creating: ade20k/images/training/
   creating: ade20k/images/training/annotations/
  inflating: ade20k/images/training/annotations/ADE_frame_00000040.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000046.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000101.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000140.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000180.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000185.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000193.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000220.png  
  inflating: ade20k/images/training/annotations/ADE_frame_00000221.png  
  inflating: ade20k/images/training/annotations

--2021-12-20 08:46:57--  http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet101-upernet/decoder_epoch_50.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162479372 (155M)
Saving to: ‘decoder_epoch_50.pth’

     0K .......... .......... .......... .......... ..........  0%  748K 3m32s
    50K .......... .......... .......... .......... ..........  0% 1.46M 2m39s
   100K .......... .......... .......... .......... ..........  0% 41.8M 1m47s
   150K .......... .......... .......... .......... ..........  0% 71.6M 81s
   200K .......... .......... .......... .......... ..........  0% 1.52M 85s
   250K .......... .......... .......... .......... ..........  0% 44.5M 71s
   300K .......... .......... .......... .......... ..........  0%  152M 61s
   350K .......... .......... .......... ...

In [ ]:
# System libs
import os
import time
# import math
import random
import argparse
from distutils.version import LooseVersion
# Numerical libs
import torch
import torch.nn as nn
# Our libs
from mit_semseg.config import cfg
from mit_semseg.dataset import TrainDataset
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import AverageMeter, parse_devices, setup_logger
from mit_semseg.lib.nn import UserScatteredDataParallel, user_scattered_collate, patch_replication_callback

In [ ]:
root_dataset = "./data/"
list_train = "./data/training.odgt"
list_val = "./data/training.odgt"
num_class = 150
padding_constant = 32
segm_downsampling_rate = 4
random_flip = True
arch_encoder="resnet101"
arch_decoder="upernet"
fc_dim=2048
weights_encoder = "encoder_epoch_50.pth"
weights_decoder = "decoder_epoch_50.pth"
batch_size_per_gpu = 2
num_epoch = 5
start_epoch = 0
epoch_iters = 50
optim = "SGD"
lr_encoder = 0.02
lr_decoder = 0.02
lr_pow = 0.9
beta1 = 0.9
weight_decay = 1e-4
deep_sup_scale = 0.4
fix_bn = False
workers = 2
disp_iter = 1
seed = 304
running_lr_encoder = ""
running_lr_encoder = ""
class dataset_custom:
  def __init__(self):
    self.imgSizes = (300, 375, 450, 525, 600)
    self.imgMaxSize = 1000
    self.padding_constant = 32
    self.segm_downsampling_rate = 4
trained_dataset = dataset_custom()
DIR = "ckpt/ade20k-resnet101-upernet"

In [ ]:
def checkpoint(nets, history, epoch):
    print('Saving checkpoints...')
    (net_encoder, net_decoder, crit) = nets

    dict_encoder = net_encoder.state_dict()
    dict_decoder = net_decoder.state_dict()

    torch.save(
        history,
        '{}/history_epoch_{}.pth'.format(DIR, epoch))
    torch.save(
        dict_encoder,
        '{}/encoder_epoch_{}.pth'.format(DIR, epoch))
    torch.save(
        dict_decoder,
        '{}/decoder_epoch_{}.pth'.format(DIR, epoch))

In [ ]:
def create_optimizers(nets):
    (net_encoder, net_decoder, crit) = nets
    '''
    optimizer_encoder = torch.optim.SGD(
        filter(lambda p: p.requires_grad, net_encoder.parameters()),
        lr=lr_encoder,
        momentum=beta1)
    '''
    optimizer_decoder = torch.optim.SGD(
        filter(lambda p: p.requires_grad, net_decoder.parameters()),
        lr=lr_decoder,
        momentum=beta1)
    return (0, optimizer_decoder)

In [ ]:
def adjust_learning_rate(optimizers, cur_iter):
    scale_running_lr = ((1. - float(cur_iter) / max_iters) ** lr_pow)
    running_lr_encoder = lr_encoder * scale_running_lr
    running_lr_decoder = lr_decoder * scale_running_lr

    (optimizer_encoder, optimizer_decoder) = optimizers
    for param_group in optimizer_decoder.param_groups:
        param_group['lr'] = running_lr_decoder

In [ ]:
def train(segmentation_module, iterator, optimizers, history, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    ave_total_loss = AverageMeter()
    ave_acc = AverageMeter()

    segmentation_module.train(not fix_bn)

    # main loop
    tic = time.time()
    for i in range(epoch_iters):
        # load a batch of data
        batch_data = next(iterator)
        data_time.update(time.time() - tic)
        segmentation_module.zero_grad()

        # adjust learning rate
        cur_iter = i + (epoch - 1) * epoch_iters
        adjust_learning_rate(optimizers, cur_iter)

        batch_data = batch_data[0]

        # forward pass
        loss, acc = segmentation_module(batch_data)
        loss = loss.mean()
        acc = acc.mean()

        # Backward
        loss.backward()
        for optimizer in optimizers:
          if(optimizer != 0):
            optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - tic)
        tic = time.time()

        # update average loss and acc
        ave_total_loss.update(loss.data.item())
        ave_acc.update(acc.data.item()*100)

        # calculate accuracy, and display
        if i % disp_iter == 0:
            print('Epoch: [{}][{}/{}], Time: {:.2f}, Data: {:.2f}, '
                  'lr_encoder: {:.6f}, lr_decoder: {:.6f}, '
                  'Accuracy: {:4.2f}, Loss: {:.6f}'
                  .format(epoch, i, epoch_iters,
                          batch_time.average(), data_time.average(),
                          running_lr_encoder, running_lr_decoder,
                          ave_acc.average(), ave_total_loss.average()))

            fractional_epoch = epoch - 1 + 1. * i / epoch_iters
            history['train']['epoch'].append(fractional_epoch)
            history['train']['loss'].append(loss.data.item())
            history['train']['acc'].append(acc.data.item())

In [ ]:
def main(gpus):
    # Network Builders
    net_encoder = ModelBuilder.build_encoder(
        arch=arch_encoder.lower(),
        fc_dim=fc_dim,
        weights=weights_encoder)
    net_decoder = ModelBuilder.build_decoder(
        arch=arch_decoder.lower(),
        fc_dim=fc_dim,
        num_class=num_class,
        weights=weights_decoder)

    crit = nn.NLLLoss(ignore_index=-1)

    segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)

    for params in segmentation_module.parameters():
      params.requires_grad = False
    segmentation_module.decoder.conv_last[1].weight.requires_grad = True
    segmentation_module.decoder.conv_last[1].bias.requires_grad = True

    # Dataset and Loader
    dataset_train = TrainDataset(
        root_dataset,
        list_train,
        trained_dataset,
        batch_per_gpu=batch_size_per_gpu)
    
    print("so far so good")

    loader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=len(gpus),  # we have modified data_parallel
        shuffle=False,  # we do not use this param
        collate_fn=user_scattered_collate,
        num_workers=workers,
        drop_last=True,
        pin_memory=True)
    print('1 Epoch = {} iters'.format(epoch_iters))

    # create loader iterator
    iterator_train = iter(loader_train)

    # load nets into gpu
    if len(gpus) > 1:
        segmentation_module = UserScatteredDataParallel(
            segmentation_module,
            device_ids=gpus)
        # For sync bn
        patch_replication_callback(segmentation_module)

    # Set up optimizers
    nets = (net_encoder, net_decoder, crit)
    optimizers = create_optimizers(nets)

    # Main loop
    history = {'train': {'epoch': [], 'loss': [], 'acc': []}}

    for epoch in range(start_epoch, num_epoch):
        train(segmentation_module, iterator_train, optimizers, history, epoch+1)

        # checkpointing
        checkpoint(nets, history, epoch+1)

    print('Training Done!')

In [ ]:
gpus = parse_devices("0")
gpus = [x.replace('gpu', '') for x in gpus]
gpus = [int(x) for x in gpus]
num_gpus = len(gpus)
batch_size = num_gpus * batch_size_per_gpu

max_iters = epoch_iters * num_epoch
running_lr_encoder = lr_encoder
running_lr_decoder = lr_decoder

random.seed(seed)
torch.manual_seed(seed)

main(gpus)

Loading weights for net_encoder
Loading weights for net_decoder
# samples: 6118
so far so good
1 Epoch = 50 iters
Epoch: [1][0/50], Time: 16.41, Data: 0.12, lr_encoder: 0.020000, lr_decoder: 0.020000, Accuracy: 9.99, Loss: 11.370049
Epoch: [1][1/50], Time: 15.63, Data: 0.06, lr_encoder: 0.020000, lr_decoder: 0.020000, Accuracy: 11.18, Loss: 10.647085


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv ckpt/ade20k-resnet101-upernet drive/MyDrive